In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import multiprocessing
from IMDBDataset import IMDBDataset, IMDBCollate
import time

import spacy
tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

In [2]:
df = pd.read_csv('movie_data.csv')
# split the dataset into train, test and validation sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [3]:
train_dataset = IMDBDataset(train_df, tokenizer, min_freq=5, vocabulary=None)
val_dataset = IMDBDataset(val_df, tokenizer, min_freq=None, vocabulary=train_dataset.vocab)
test_dataset = IMDBDataset(test_df, tokenizer, min_freq=None, vocabulary=train_dataset.vocab)

In [4]:
train_dataloader = DataLoader(train_dataset, 
                              batch_size=32, 
                              shuffle=True,
                              collate_fn=IMDBCollate(pad_idx=train_dataset.vocab['<PAD>']), 
                              num_workers=multiprocessing.cpu_count())
val_dataloader = DataLoader(val_dataset, 
                            batch_size=32, 
                            shuffle=False, 
                            collate_fn=IMDBCollate(pad_idx=train_dataset.vocab['<PAD>']), 
                            num_workers=multiprocessing.cpu_count())
test_dataloader = DataLoader(test_dataset, 
                             batch_size=32, 
                             shuffle=False, 
                             collate_fn=IMDBCollate(pad_idx=train_dataset.vocab['<PAD>']), 
                             num_workers=multiprocessing.cpu_count())

### General Settings

In [5]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = train_dataset.vocab.__len__()
LEARNING_RATE = 0.005
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 2

### Model

In [6]:
class RNN(torch.nn.Module):

    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        # self.rnn = torch.nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text) # text = [batch size, sentence length] -> embedded = [batch size, sentence length, embedding dim]
        # output, (hidden, cell) = self.rnn(embedded) # output = [batch size, sent len, hid dim], hidden = [num_layer=1, batch size, hid dim], cell = [num_layer=1, batch size, hid dim]
        output, hidden = self.rnn(embedded) # output = [batch size, sent len, hid dim], hidden = [num_layer=1, batch size, hid dim], cell = [num_layer=1, batch size, hid dim]
        return self.fc(hidden.squeeze(0)) # hidden = [num_layer=1, batch size, hid dim] -> hidden.squeeze(0) = [batch size, hid dim]

In [7]:
# embedding = torch.nn.Embedding(VOCABULARY_SIZE, EMBEDDING_DIM)
# rnn = torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True)

In [8]:
# for batch in train_dataloader:
#     text, label = batch
#     embedded = embedding(text)
#     output, (hidden, cell) = rnn(embedded)
#     print('text shape: ', text.shape)
#     print('embedded shape: ', embedded.shape)
#     print('output shape: ', output.shape)
#     print('hidden shape: ', hidden.shape)
#     print('cell shape: ', cell.shape)
#     break

In [9]:
model = RNN(input_dim=VOCABULARY_SIZE,
            embedding_dim=EMBEDDING_DIM, 
            hidden_dim=HIDDEN_DIM, 
            output_dim=NUM_CLASSES)

In [10]:
model.to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

In [11]:
## count trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [12]:
count_parameters(model)

5601538

In [13]:
def compute_accuracy(model, data_loader, device):
    with torch.no_grad():
        correct_pred, num_examples = 0, 0
        for i, (features, targets) in enumerate(data_loader):
            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [14]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_dataloader):
        text, labels = batch_data
        text = text.to(DEVICE)
        labels = labels.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = loss_fn(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_dataloader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_dataloader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, val_dataloader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_dataloader, DEVICE):.2f}%')

Epoch: 001/015 | Batch 000/1125 | Loss: 0.7213
Epoch: 001/015 | Batch 050/1125 | Loss: 0.7217
Epoch: 001/015 | Batch 100/1125 | Loss: 0.6919
Epoch: 001/015 | Batch 150/1125 | Loss: 0.6629
Epoch: 001/015 | Batch 200/1125 | Loss: 0.6295
Epoch: 001/015 | Batch 250/1125 | Loss: 0.8206
Epoch: 001/015 | Batch 300/1125 | Loss: 0.7054
Epoch: 001/015 | Batch 350/1125 | Loss: 0.6780
Epoch: 001/015 | Batch 400/1125 | Loss: 0.7195
Epoch: 001/015 | Batch 450/1125 | Loss: 0.7121
Epoch: 001/015 | Batch 500/1125 | Loss: 0.7643
Epoch: 001/015 | Batch 550/1125 | Loss: 0.7188
Epoch: 001/015 | Batch 600/1125 | Loss: 0.8829
Epoch: 001/015 | Batch 650/1125 | Loss: 0.7059
Epoch: 001/015 | Batch 700/1125 | Loss: 0.6057
Epoch: 001/015 | Batch 750/1125 | Loss: 0.6217
Epoch: 001/015 | Batch 800/1125 | Loss: 0.7067
Epoch: 001/015 | Batch 850/1125 | Loss: 0.7004
Epoch: 001/015 | Batch 900/1125 | Loss: 0.6920
Epoch: 001/015 | Batch 950/1125 | Loss: 0.7005
Epoch: 001/015 | Batch 1000/1125 | Loss: 0.7276
Epoch: 001/0

KeyboardInterrupt: 

In [ ]:
def compute_accuracy_2(model, data_loader, device):
    with torch.no_grad():
        correct_pred, num_examples = 0, 0
        for i, (features, targets) in enumerate(data_loader):
            features = features.to(device)
            targets = targets.float().to(device)
            print(targets)
            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)
            print(targets, predicted_labels, end='\n\n')

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [ ]:
compute_accuracy_2(model, train_dataloader, DEVICE)